In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from utils import extract_for_ensemble, create_matrix_from_raw, RAND_SEED
from models import IRSVD, Baseline, GBias, SVP, SVT, RSVD

## KFolds
- Split data into 10 folds (90% train set)
- Use random state for reproducibility

In [ ]:
data_pd = pd.read_csv("./data/data_train.csv")
kf = KFold(n_splits=10, shuffle=True, random_state=RAND_SEED)
# Check whether we have the same splits
for train_set, test_set in kf.split(data_pd):
    print(train_set)
    print(test_set)

## Models
- Predict matrix for different models and parameters
- Save the produced matrix for ensemble (for all folds)
- Also train on entire dataset

### Improved Regularized SVD

- For ensemble training

In [ ]:
params = (
    ("mean", 96, 0.01, 0.02, 0.05, 13),
    ("mean", 148, 0.01, 0.02, 0.05, 13),
    ("mean", 296, 0.01, 0.02, 0.05, 14),
    ("mean", 324, 0.01, 0.02, 0.05, 15),
    ("zero", 96, 0.01, 0.02, 0.05, 13),
    ("zero", 148, 0.01, 0.02, 0.05, 13),
    ("zero", 296, 0.01, 0.02, 0.05, 14),
    ("zero", 324, 0.01, 0.02, 0.05, 15),
)

for idx, (train_set, test_set) in enumerate(kf.split(data_pd)):
    train_data = data_pd.iloc[train_set]
    test_data = data_pd.iloc[test_set]
    
    train_matrix = create_matrix_from_raw(train_data)
    test_matrix = create_matrix_from_raw(test_data)
    
    for param in params:
        biases, features, eta, lambda1, lambda2, epochs = param
        fname = "irsvd_"+biases+"_"+str(features)
        print(param)
        model = IRSVD(train_matrix, biases=biases, features=features,
                      eta=eta, lambda1=lambda1, lambda2=lambda2, epochs=epochs)
        print(model.train(test_matrix=test_matrix))
        rec_matrix = model.reconstruct_matrix()
        extract_for_ensemble(rec_matrix, fname, idx+1, train=True)

- Entire dataset training

In [ ]:
params = (
    ("mean", 96, 0.01, 0.02, 0.05, 13),
    ("mean", 148, 0.01, 0.02, 0.05, 13),
    ("mean", 296, 0.01, 0.02, 0.05, 14),
    ("mean", 324, 0.01, 0.02, 0.05, 15),
    ("zero", 96, 0.01, 0.02, 0.05, 13),
    ("zero", 148, 0.01, 0.02, 0.05, 13),
    ("zero", 296, 0.01, 0.02, 0.05, 14),
    ("zero", 324, 0.01, 0.02, 0.05, 15),
)

train_matrix = create_matrix_from_raw(data_pd)
for param in params:
    biases, features, eta, lambda1, lambda2, epochs = param
    fname = "irsvd_"+biases+"_"+str(features)
    print(param)
    model = IRSVD(train_matrix, biases=biases, features=features,
                    eta=eta, lambda1=lambda1, lambda2=lambda2, epochs=epochs)
    print(model.train())
    rec_matrix = model.reconstruct_matrix()
    extract_for_ensemble(rec_matrix, fname, 0, train=False)

### Baseline

- For ensemble training

In [ ]:
params = (
    (3, 0.1, 3),
)

for idx, (train_set, test_set) in enumerate(kf.split(data_pd)):
    train_data = data_pd.iloc[train_set]
    test_data = data_pd.iloc[test_set]
    
    train_matrix = create_matrix_from_raw(train_data)
    test_matrix = create_matrix_from_raw(test_data)
    
    for param in params:
        K, lambda1, epochs = param
        fname = "baseline_"+str(K)+"_"+str(epochs)
        print(param)
        model = Baseline(train_matrix, K=K, lambda1=lambda1, epochs=epochs)
        print(model.train(test_matrix=test_matrix))
        rec_matrix = model.reconstruct_matrix()
        extract_for_ensemble(rec_matrix, fname, idx+1, train=True)

- Entire dataset training

In [ ]:
params = (
    (3, 0.1, 3),
)

train_matrix = create_matrix_from_raw(data_pd)
print(data_pd.shape)
for param in params:
    K, lambda1, epochs = param
    fname = "baseline_"+str(K)+"_"+str(epochs)
    print(param)
    model = Baseline(train_matrix, K=K, lambda1=lambda1, epochs=epochs)
    print(model.train())
    rec_matrix = model.reconstruct_matrix()
    extract_for_ensemble(rec_matrix, fname, 0, train=False)

### Global biases

- For ensemble training

In [ ]:
params = (
    (0.001, 5),
)

for idx, (train_set, test_set) in enumerate(kf.split(data_pd)):
    train_data = data_pd.iloc[train_set]
    test_data = data_pd.iloc[test_set]
    
    train_matrix = create_matrix_from_raw(train_data)
    test_matrix = create_matrix_from_raw(test_data)
    
    for param in params:
        lambda1, epochs = param
        fname = "global_"+str(epochs)
        print(param)
        model = GBias(train_matrix, lambda1=lambda1, epochs=epochs)
        print(model.train(test_matrix=test_matrix))
        rec_matrix = model.reconstruct_matrix()
        extract_for_ensemble(rec_matrix, fname, idx+1, train=True)

- Entire dataset training

In [ ]:
params = (
    (0.001, 5),
)

train_matrix = create_matrix_from_raw(data_pd)
for param in params:
    lambda1, epochs = param
    fname = "global_"+str(epochs)
    print(param)
    model = GBias(train_matrix, lambda1=lambda1, epochs=epochs)
    print(model.train())
    rec_matrix = model.reconstruct_matrix()
    extract_for_ensemble(rec_matrix, fname, 0, train=False)

### SVProjection

- For ensemble training

In [ ]:
params = (
    (5, 3, 10),
)

for idx, (train_set, test_set) in enumerate(kf.split(data_pd)):
    train_data = data_pd.iloc[train_set]
    test_data = data_pd.iloc[test_set]
    
    train_matrix = create_matrix_from_raw(train_data)
    test_matrix = create_matrix_from_raw(test_data)
    
    for param in params:
        eta, K, epochs = param
        fname = "svp_"+str(eta)+"_"+str(K)+"_"+str(epochs)
        print(param)
        model = SVP(train_matrix, eta=eta, K=K, epochs=epochs)
        print(model.train(test_matrix=test_matrix))
        rec_matrix = model.reconstruct_matrix()
        extract_for_ensemble(rec_matrix, fname, idx+1, train=True)

- Entire dataset training

In [ ]:
params = (
    (5, 3, 10),
)

train_matrix = create_matrix_from_raw(data_pd)
for param in params:
    eta, K, epochs = param
    fname = "svp_"+str(eta)+"_"+str(K)+"_"+str(epochs)
    print(param)
    model = SVP(train_matrix, eta=eta, K=K, epochs=epochs)
    print(model.train())
    rec_matrix = model.reconstruct_matrix()
    extract_for_ensemble(rec_matrix, fname, 0, train=False)

### Nuclear norm relaxation / SVT

- For ensemble training

In [ ]:
params = (
    (1.2, 2000, 28),
    (1.2, 1000, 15),
    (1.2, 1000, 15),
    (1.5, 2000, 22),
)

for idx, (train_set, test_set) in enumerate(kf.split(data_pd)):
    train_data = data_pd.iloc[train_set]
    test_data = data_pd.iloc[test_set]
    
    train_matrix = create_matrix_from_raw(train_data)
    test_matrix = create_matrix_from_raw(test_data)
    
    for param in params:
        eta, tau, epochs = param
        fname = "svt_"+str(eta)+"_"+str(tau)+"_"+str(epochs)
        print(param)
        model = SVT(train_matrix, eta=eta, tau=tau, epochs=epochs)
        print(model.train(test_matrix=test_matrix))
        rec_matrix = model.reconstruct_matrix()
        extract_for_ensemble(rec_matrix, fname, idx+1, train=True)

- Entire dataset training

In [ ]:
params = (
    (1.2, 2000, 28),
    (1.2, 1000, 15),
    (1.2, 1000, 15),
    (1.5, 2000, 22),
)

train_matrix = create_matrix_from_raw(data_pd)
for param in params:
    eta, tau, epochs = param
    fname = "svt_"+str(eta)+"_"+str(tau)+"_"+str(epochs)
    print(param)
    model = SVT(train_matrix, eta=eta, tau=tau, epochs=epochs)
    print(model.train())
    rec_matrix = model.reconstruct_matrix()
    extract_for_ensemble(rec_matrix, fname, 0, train=False)

### Regularized SVD

- For ensemble training

In [ ]:
params = (
    (96, 0.01, 0.02, 13),
)

for idx, (train_set, test_set) in enumerate(kf.split(data_pd)):
    train_data = data_pd.iloc[train_set]
    test_data = data_pd.iloc[test_set]
    
    train_matrix = create_matrix_from_raw(train_data)
    test_matrix = create_matrix_from_raw(test_data)
    
    for param in params:
        features, eta, lambda1, epochs = param
        fname = "rsvd_"+str(features)+"_"+str(epochs)
        print(param)
        model = RSVD(train_matrix, features=features,
                      eta=eta, lambda1=lambda1, epochs=epochs)
        print(model.train(test_matrix=test_matrix))
        rec_matrix = model.reconstruct_matrix()
        extract_for_ensemble(rec_matrix, fname, idx+1, train=True)

- Entire dataset training

In [ ]:
params = (
    (96, 0.01, 0.02, 13),
)

train_matrix = create_matrix_from_raw(data_pd)
for param in params:
    features, eta, lambda1, epochs = param
    fname = "rsvd_"+str(features)+"_"+str(epochs)
    print(param)
    model = RSVD(train_matrix, features=features,
                    eta=eta, lambda1=lambda1, epochs=epochs)
    print(model.train())
    rec_matrix = model.reconstruct_matrix()
    extract_for_ensemble(rec_matrix, fname, 0, train=False)